In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


In [ ]:
df = pd.read_csv('../input/9309a1dataset/recipe_data.csv')
# frequent for every feature
top_5_preparation = df.groupby('User_ID')['Preparation_Steps'].value_counts().groupby(level=0, group_keys=False).head(5)
top_5_ingredient = df.groupby('User_ID')['Ingredients_List'].value_counts().groupby(level=0, group_keys=False).head(5)
top_5_type = df.groupby('User_ID')['Cuisine_Type'].value_counts().groupby(level=0, group_keys=False).head(5)

# most frequent
top_1_preparation = df.groupby('User_ID')['Preparation_Steps'].value_counts().groupby(level=0, group_keys=False).head(1).reset_index(name='count')
top_1_ingredient = df.groupby('User_ID')['Ingredients_List'].value_counts().groupby(level=0, group_keys=False).head(1).reset_index(name='count')
top_1_type = df.groupby('User_ID')['Cuisine_Type'].value_counts().groupby(level=0, group_keys=False).head(1).reset_index(name='count')

top_1_preparation = top_1_preparation.rename(columns={'Preparation_Steps': 'Top_Pre'})
top_1_ingredient = top_1_ingredient.rename(columns={'Ingredients_List': 'Top_Ing'})
top_1_type = top_1_type.rename(columns={'Cuisine_Type': 'Top_Type'})

top_5_type_df = top_5_type.reset_index(name='Type_Count')
top_5_preparation_df = top_5_preparation.reset_index(name='Pre_Count')
top_5_ingredient_df = top_5_ingredient.reset_index(name='Ing_Count')


user_total = df.groupby('User_ID').size()
user_total_df = user_total.reset_index(name='Total_Time')

df = pd.merge(df, top_5_type_df[['User_ID', 'Cuisine_Type', 'Type_Count']], on=['User_ID', 'Cuisine_Type'], how='left')
df = pd.merge(df, top_5_preparation_df[['User_ID', 'Preparation_Steps', 'Pre_Count']], on=['User_ID', 'Preparation_Steps'], how='left')
df = pd.merge(df, top_5_ingredient_df[['User_ID', 'Ingredients_List', 'Ing_Count']], on=['User_ID', 'Ingredients_List'], how='left')

df = pd.merge(df, top_1_type[['User_ID', 'Top_Type']], on=['User_ID'], how='left')
df = pd.merge(df, top_1_preparation[['User_ID', 'Top_Pre']], on=['User_ID'], how='left')
df = pd.merge(df, top_1_ingredient[['User_ID', 'Top_Ing']], on=['User_ID'], how='left')

df = pd.merge(df, user_total_df[['User_ID', 'Total_Time']], on=['User_ID'], how='left')

print(df.describe())

In [ ]:
def cal_ing_freq(row):
    return row['Ing_Count'] *1.0 / row['Total_Time']

def cal_pre_freq(row):
    return row['Pre_Count'] *1.0 / row['Total_Time']

def cal_type_freq(row):
    return row['Type_Count'] *1.0 / row['Total_Time']

# Apply the function to each row
df['Type_Freq'] = df.apply(cal_type_freq, axis=1)
df['Pre_Freq'] = df.apply(cal_pre_freq, axis=1)
df['Ing_Freq'] = df.apply(cal_ing_freq, axis=1)

print(df.head(1))

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = text.lower().replace(',', '').split()
    return [word for word in tokens if word not in stop_words]

In [ ]:
ing_sentences = [preprocess(sentence) for sentence in df['Ingredients_List'].tolist() + df['Top_Ing'].tolist()]
ing_model = Word2Vec(ing_sentences, vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = preprocess(sentence)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

def calculate_similarity(row, model, c1, c2):
    vector1 = get_sentence_vector(row[c1], model)
    vector2 = get_sentence_vector(row[c2], model)
    similarity = cosine_similarity([vector1], [vector2])
    return similarity[0][0]

# Apply the function to calculate similarity for each row
df['Ing_Sim'] = df.apply(calculate_similarity, axis=1, model=ing_model,c1='Ingredients_List', c2='Top_Ing')

In [ ]:
pre_sentences = [preprocess(sentence) for sentence in df['Preparation_Steps'].tolist() + df['Top_Pre'].tolist()]
pre_model = Word2Vec(pre_sentences, vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = preprocess(sentence)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Apply the function to calculate similarity for each row
df['Pre_Sim'] = df.apply(calculate_similarity, axis=1, model=pre_model, c1='Preparation_Steps', c2='Top_Pre')

In [ ]:
print(df.columns)

In [ ]:
# check Allergen_Information

def check_aller(row):
    index = str(row['Ingredients_List']).find(str(row['Allergen_Information']))
    if index == -1:
        return 0
    return 1
df['Is_Aller'] = df.apply(check_aller, axis=1)

In [ ]:
print(df.head(10))

In [ ]:
# df.to_csv('/kaggle/working/recipe_processed.csv')

In [ ]:
df['Cooking_Time_Minutes'].hist(bins=100, figsize=(10, 6))
plt.xlabel('Minutes ') 
plt.ylabel('Density') 
plt.show()

In [ ]:
df['User_Rating'].hist(bins=12, figsize=(10, 6))
plt.xlabel('Rating') 
plt.ylabel('Density') 
plt.show()

In [ ]:
 
df['Serving_Size'].hist(bins=12, figsize=(10, 6))
plt.xlabel('People Amount') 
plt.ylabel('Density') 
plt.show()

In [ ]:
 

df['Calories_Per_Serving'].hist(bins=12, figsize=(10, 6))
plt.xlabel('Calory') 
plt.ylabel('Density') 
plt.show()

In [ ]:
df['Cost_Per_Serving'].hist(bins=30, figsize=(10, 6))
plt.xlabel('$') 
plt.ylabel('Density') 
plt.show()

In [ ]:
df['Popularity_Score'].hist(bins=500, figsize=(10, 6))
plt.xlabel('Popularity') 
plt.ylabel('Density') 
plt.show()
 

In [ ]:
df['Seasonal_Availability'].hist(bins=3, figsize=(10, 6))
plt.xlabel('Type') 
plt.ylabel('Density') 
plt.show()

In [ ]:
import math

def wilson_score(row):
    
    z = 1.96
    x = row['User_Rating']
    n = row['Popularity_Score']
    if n == 0:
        return 0.0  # 如果没有评分，返回 0 的估计值
    p_hat = x / n
    denominator = 1 + (z**2 / n)
    center_adjusted_probability = p_hat + (z**2 / (2 * n))
    adjusted_stddev = z * math.sqrt((p_hat * (1 - p_hat) / n) + (z**2 / (4 * n**2)))
    
    lower_bound = (center_adjusted_probability - adjusted_stddev) / denominator
    upper_bound = (center_adjusted_probability + adjusted_stddev) / denominator
    return (lower_bound + upper_bound) / 2

df['wilson_score'] = df.apply(wilson_score, axis=1)
print(df.head(10))

In [ ]:
from sklearn.metrics import mean_squared_error

def mean_absolute_error(y_true, y_pred):
    return sum(abs(y - y_hat) for y, y_hat in zip(y_true, y_pred)) / len(y_true)


def getResult(x_train, y_train, x_test, y_test, model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_pred, y_test)
    print('mse: ', mse)
    print('rmse: ', rmse)
    print('mae: ', mae)



In [ ]:
print(df.columns)

In [ ]:
df1 = df[['Cooking_Time_Minutes', 'User_Rating', 'Serving_Size', 'Calories_Per_Serving', 'Cost_Per_Serving', 'Popularity_Score', 'Cuisine_Type', 'Difficulty_Level', 'Occasion', 'Seasonal_Availability', 'Type_Freq', 'Pre_Freq', 'Ing_Freq']]

In [ ]:
dataset2_dummy = pd.get_dummies(df1, columns=['Cuisine_Type', 'Difficulty_Level', 'Occasion', 'Seasonal_Availability'])
target = dataset2_dummy['User_Rating']
features = dataset2_dummy.drop(columns=['User_Rating'])

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split



rf_regressor = RandomForestRegressor(
    n_estimators=100,          # Number of trees
    max_depth=None,             # No limit on the depth of trees
    min_samples_split=2,        # Minimum samples required to split an internal node
    min_samples_leaf=1,         # Minimum samples required to be at a leaf node
    max_features='auto',        # Consider all features when splitting nodes
    bootstrap=True,             # Use bootstrap sampling
    random_state=42            # Ensure reproducibility
)


gbdt = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
ridge_regressor = Ridge(alpha=1.0)
lasso = Lasso(alpha=1.0) 
xgb_regressor = xgb.XGBRegressor(
    n_estimators=100,          # Number of trees
    learning_rate=0.1,         # Learning rate
    max_depth=5,               # Depth of each tree
    min_child_weight=1,        # Minimum sum of instance weight needed in a child
    subsample=0.8,             # Fraction of samples used for fitting trees
    colsample_bytree=0.8,      # Fraction of features used for each tree
    gamma=0,                   # Minimum loss reduction for pruning
    reg_alpha=0,               # L1 regularization
    reg_lambda=1,              # L2 regularization
    objective='reg:squarederror',  # Standard regression objective
    random_state=42
)

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)


In [ ]:
getResult(x_train, y_train, x_test, y_test, rf_regressor)
getResult(x_train, y_train, x_test, y_test, gbdt)
getResult(x_train, y_train, x_test, y_test, ridge_regressor)
getResult(x_train, y_train, x_test, y_test, lasso)
getResult(x_train, y_train, x_test, y_test, xgb_regressor)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


dnn = Sequential()
dnn.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
dnn.add(Dense(64, activation='relu'))
dnn.add(Dense(32, activation='relu'))
dnn.add(Dense(1))

dnn.compile(optimizer='adam', loss='mean_squared_error')
getResult(x_train, y_train, x_test, y_test, dnn)

def getDNNResult(x_train, y_train, x_test, y_test):
    dnn.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_pred, y_test)
    print('mse: ', mse)
    print('rmse: ', rmse)
    print('mae: ', mae)